In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import itertools
import random

from dask.distributed import Client, progress
import dask
import dask.dataframe as dd
#client.shutdown()
#client.close()
dask.config.set({'temporary_directory': '/pasteur/sonic/scratch/public/cduitama/RascovanProject/tmp/'})
#dask.config.set({"optimization.fuse.ave-width": 5})
client = Client()
client

# Classification

## Data preparation

In [ ]:
imatrices=dict()
ikmers=dict()

In [ ]:
SRR_aSGenomes_HS_Oral=pd.read_csv("Metadata/aSGenomes/aSGenomes_HS_Oral_SRR.txt",header=None,names=["SRR_accession"])
SRR_mMetagenomes_HS_Oral=pd.read_csv("Metadata/mMetagenomes/Oral_HS_mMetagenomes_SRR.txt",\
                                     header=None,names=["SRR_accession"])

HS_Oral_metadata=pd.read_csv("Metadata/aMetagenomes/HS_Oral_metadata.csv")
SRR_HS_Oral=["Kmer"]+list(HS_Oral_metadata["SRR_accession"])

SRR_HS_Oral_aMetagenomes=SRR_HS_Oral[1:]
SRR_HS_Oral_aSGenomes=list(SRR_aSGenomes_HS_Oral.SRR_accession)
SRR_HS_Oral_mMetagenomes=list(SRR_mMetagenomes_HS_Oral.SRR_accession)
names=SRR_HS_Oral_aMetagenomes+SRR_HS_Oral_aSGenomes+SRR_HS_Oral_mMetagenomes


In [ ]:
len(names)

In [ ]:
to_take=np.genfromtxt("Classification/list_of_indeces_of_RepresentativeSample.csv",delimiter=",")

print(len(to_take))


In [ ]:
combined_total=dd.read_table("kmMatrices/combination/combination_whole_matrix.txt",sep=" ",header=None,names=names)
all_kmers=combined_total.Kmer.compute()
all_kmers=all_kmers.reset_index()
all_kmers=all_kmers.drop("index",axis=1)
ikmers["RepresentativeSample"]=all_kmers.iloc[to_take]
icombined_total=combined_total[combined_total["Kmer"].isin(ikmers["RepresentativeSample"]["Kmer"])]
imatrices["RepresentativeSample"]=icombined_total.drop("Kmer",axis=1).values

imatrices["RepresentativeSample"]=imatrices["RepresentativeSample"].persist()

from sklearn.preprocessing import Binarizer
ecombined_total=imatrices["RepresentativeSample"]
b_ecombined_total = Binarizer().fit_transform(ecombined_total)  # fit does nothing.

import dask.array as da
from sklearn.decomposition import PCA
#Here I convert the np array into a Dask array using chunks of 1 GB
dX_15 = b_ecombined_total.transpose()
pca_15 = PCA(n_components=2)
to_plot_15=pca_15.fit(dX_15)

np.savetxt("Preprocessed_data/combination/RepresentativeSample_Binarization_PCA.txt", to_plot_15.components_, delimiter=',')

print(to_plot_15.explained_variance_ratio_)

print(to_plot_15.components_.shape)

In [ ]:
from numpy import savetxt
savetxt("Classification/RepresentativeSample_binarized.csv", b_ecombined_total, delimiter=',')

attempt=dd.read_table("Classification/RepresentativeSample_binarized.csv",delimiter=",").values

b_ecombined_total.shape

attempt=attempt.compute()

In [ ]:
#1=Ancient
#2=NonAncient
to_take=one+two+three+four+five+six+seven
labels=[]
labels=labels+[False]*len(one)
labels=labels+[False]*len(two)
labels=labels+[True]*len(three)
labels=labels+[False]*len(four)
labels=labels+[True]*len(five)
labels=labels+[False]*len(six)
labels=labels+[True]*len(seven)
Data = pd.DataFrame(list(zip(to_take, labels)),
               columns =['Index', 'Label'])
Data.sort_values(by='Index' , inplace=True)
Data=Data.reset_index()

Data=Data.drop("index",axis=1)
Data.head()

np.savetxt("Classification/Labels_RepresentativeSample.csv",Data["Label"], delimiter=',')



In [ ]:
labels_RepresentativeSample=np.genfromtxt("Classification/Labels_RepresentativeSample.csv",delimiter=",")

## Data splitting

In [ ]:
import joblib
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(n_jobs=-1)
with joblib.parallel_backend('dask'):
    LR.fit(X_train, y_train)
    y_test_predicted_LR=LR.predict(X_test)
    y_train_predicted_LR=LR.predict(X_train)

In [ ]:
X_train.shape

y_train.shape

X_test.shape

y_test.shape

# Logistic regression

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_train_predicted_LR)

import numpy as np
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_test_predicted_LR)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_train,y_train_predicted_LR)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_test_predicted_LR)

len(np.where(labels_RepresentativeSample==False)[0])

len(np.where(labels_RepresentativeSample==True)[0])

len(np.where(y_train!=y_train_predicted_LR)[0])

len(np.where(y_train==y_train_predicted_LR)[0])

y_train_score_LR=LR.predict_proba(X_train)[:,1]

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_train, y_train_score_LR)

fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve for training set (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_train, y_train_score_LR)

# Evaluating model performance at various thresholds
df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr
}, index=thresholds)
df.index.name = "Thresholds"
df.columns.name = "Rate"

fig_thresh = px.line(
    df, title='TPR and FPR at every threshold for training set',
    width=700, height=500
)

fig_thresh.update_yaxes(scaleanchor="x", scaleratio=1)
fig_thresh.update_xaxes(range=[0, 1], constrain='domain')
fig_thresh.show()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_predicted_LR)

In [ ]:
y_test_score_LR=LR.predict_proba(X_test)[:,1]
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(y_test, y_test_score_LR)

fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve for test set (AUC={auc(fpr, tpr):.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_test_score_LR)

# Evaluating model performance at various thresholds
df = pd.DataFrame({
    'False Positive Rate': fpr,
    'True Positive Rate': tpr
}, index=thresholds)
df.index.name = "Thresholds"
df.columns.name = "Rate"

fig_thresh = px.line(
    df, title='TPR and FPR at every threshold for test set',
    width=700, height=500
)

fig_thresh.update_yaxes(scaleanchor="x", scaleratio=1)
fig_thresh.update_xaxes(range=[0, 1], constrain='domain')
fig_thresh.show()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_test_predicted_LR)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_train,y_train_predicted_LR,target_names=["Non-ancient","Ancient"]))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_test_predicted_LR,target_names=["Non-ancient","Ancient"]))

In [ ]:
np.savetxt("Classification/LR/y_test_predicted_labels.csv",y_test_predicted_LR,delimiter=",")
np.savetxt("Classification/LR/y_train_predicted_labels.csv",y_train_predicted_LR,delimiter=",")
np.savetxt("Classification/LR/y_test_true_labels.csv",y_test,delimiter=",")
np.savetxt("Classification/LR/y_train_true_labels.csv",y_train,delimiter=",")
np.savetxt("Classification/LR/y_train_score_LR.csv",y_train_score_LR,delimiter=",")
np.savetxt("Classification/LR/y_test_score_LR.csv",y_test_score_LR,delimiter=",")